In [1]:
import os
import sys
from pathlib import Path

os.chdir(Path(os.getcwd()).parents[0])
sys.path.append(os.getcwd())

import glob
import numpy as np
import torch
import pandas as pd
from scipy.stats import pearsonr

def left_align_facet_plot_titles(fig):

    ## figure out number of columns in each facet
    facet_col_wrap = len(np.unique([a['x'] for a in fig.layout.annotations]))

    # x x x x
    # x x x x <-- then these annotations
    # x x     <-- these annotations are created first

    ## we need to know the remainder
    ## because when we iterate through annotations
    ## they need to know what column they are in 
    ## (and annotations natively contain no such information)

    remainder = len(fig.data) % facet_col_wrap
    number_of_full_rows = len(fig.data) // facet_col_wrap

    annotations = fig.layout.annotations

    xaxis_col_strings = list(range(1, facet_col_wrap+1))
    xaxis_col_strings[0] = ''
    x_axis_start_positions = [fig.layout[f'xaxis{i}']['domain'][0] for i in xaxis_col_strings]

    if remainder == 0:
        x_axis_start_positions_iterator = x_axis_start_positions*number_of_full_rows
    else:
        x_axis_start_positions_iterator = x_axis_start_positions[:remainder] + x_axis_start_positions*number_of_full_rows

    for a, x in zip(annotations, x_axis_start_positions_iterator):
        a['x'] = x
        a['xanchor'] = 'left'
    fig.layout.annotations = annotations
    return fig

/home/l727n/anaconda3/envs/xai-eval/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import Eval Results

In [2]:
# File loading per dataset

file_image_inet = "/Image/eval_imagenet_dataset.npz"
file_image_oct = "/Image/eval_oct_dataset.npz"
file_image_r45 = "/Image/eval_resisc45_dataset.npz"

file_voxel_adr = "/Voxel/eval_AdrenalMNIST3D_dataset_2.npz"
file_voxel_org = "/Voxel/eval_OrganMNIST3D_dataset_2.npz"
file_voxel_ves = "/Voxel/eval_VesselMNIST3D_dataset_2.npz"

file_pc_coma = "/Point_Cloud/eval_coma_dataset_2.npz"
file_pc_m40 = "/Point_Cloud/eval_modelnet40_dataset_2.npz"
file_pc_shpn = "/Point_Cloud/eval_shapenet_dataset_2.npz"

file_loc = os.getcwd() + "/data/evaluation"

file = np.load(file_loc + file_image_inet, allow_pickle=True)
arr_image_inet = [file['arr_0'], file['arr_1'], file['arr_2']]
file = np.load(file_loc + file_image_oct, allow_pickle=True)
arr_image_oct = [file['arr_0'], file['arr_1'], file['arr_2']]
file = np.load(file_loc + file_image_r45, allow_pickle=True)
arr_image_r45 = [file['arr_0'], file['arr_1'], file['arr_2']]

file = np.load(file_loc + file_voxel_adr, allow_pickle=True)
arr_voxel_adr = [file['arr_0'], file['arr_1'], file['arr_2']]
file = np.load(file_loc + file_voxel_org, allow_pickle=True)
arr_voxel_org = [file['arr_0'], file['arr_1'], file['arr_2']]
file = np.load(file_loc + file_voxel_ves, allow_pickle=True)
arr_voxel_ves = [file['arr_0'], file['arr_1'], file['arr_2']]

file = np.load(file_loc + file_pc_coma, allow_pickle=True)
arr_pc_coma = [file['arr_0'], file['arr_1'], file['arr_2']]
file = np.load(file_loc + file_pc_m40, allow_pickle=True)
arr_pc_m40 = [file['arr_0'], file['arr_1'], file['arr_2']]
file = np.load(file_loc + file_pc_shpn, allow_pickle=True)
arr_pc_shpn = [file['arr_0'], file['arr_1'], file['arr_2']]

In [ ]:
# Fix bug in PC
for m in range(3):
    for j in range(50):
        for i in range(arr_pc_shpn[m].shape[0]):
            arr_pc_shpn[m][i,11,:][j][341] = arr_pc_shpn[m][i,11,:][j][341][:20]

# Fix bug in 3D
for m in range(3):
    for j in range(50):
        for i in range(arr_voxel_ves[m].shape[0]):
            arr_voxel_ves[m][i,12,:][j][343] = arr_voxel_ves[m][i,12,:][j][49][20:40]
            arr_voxel_ves[m][i,12,:][j][49] = arr_voxel_ves[m][i,12,:][j][49][0:20]

            print(len(arr_voxel_ves[m][i,12,:][j][49]))
            print(len(arr_voxel_ves[m][i,12,:][j][343]))

np.savez(
    os.getcwd() + "/data/evaluation/Point_Cloud/eval_shapenet_dataset_2.npz",
    arr_pc_shpn[0],
    arr_pc_shpn[1],
    arr_pc_shpn[2],
)

### Visualize Eval Results

In [3]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

data = arr_image_inet

def NormalizeData(data, min, max):
    return (data - min) / ((max - min) + 0.00000000001)

titles = ["<b>[F]</b> Faithfulness Correlation \u2191", "[F] Faithfulness Estimate \u2191", "[F] Monotonicity Correlation \u2191", "[F] Pixel Flipping (AUC) \u2193", "[F] Region Perturbation (AUC) \u2191", "[F] Insertion \u2191", "[F] Deletion \u2193", "[F] IROF (AOC) \u2191", "[F] ROAD (AUC) \u2193", "[F] Sufficiency \u2191", "<b>[R]</b> Local Lipschitz Estimate \u2193", 
           "[R] Max Sensitivity \u2193", "[R] Continuity (PCC) \u2193", "[R] Relative Input Stability \u2193", "[R] Relative Output Stability \u2193", "[R] Relative Repr. Stability \u2193", "[R] Infidelity \u2193", "<b>[C]</b> Sparseness \u2191", "[C] Complexity \u2193", "[C] Effective Complexity \u2193"]

methods = ["OC","LI","KS","SA","IxG", "GB","GC","SC","C+", "IG", "EG", "DL", "DLS", "LRP", "RA", "RoA", "LA"] if data[0].shape[1] == 20 else ["OC","LI","KS","SA","IxG", "GB", "IG", "EG", "DL", "DLS", "LRP", "RA", "RoA", "LA"]

fig = make_subplots(
    rows=5,
    cols=4,
    vertical_spacing = 0.05,
    horizontal_spacing= 0.03,
    subplot_titles=titles if data[0].shape[1] == 20 else titles[:7] + titles[7+1:],

)

plot_type =[0,0,0,1,1,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0] if data[0].shape[1] == 20 else [0,0,0,1,1,0,0,1,0,0,0,2,0,0,0,0,0,0,0]
plot_row = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5]
plot_col = [1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4]
plot_x = ["attr","attr","attr","attr","attr","attr","attr","attr","attr","attr","attr","attr","attr","attr","atten","atten","atten",] if data[0].shape[1] == 20 else ["attr","attr","attr","attr","attr","attr","attr","attr","attr","attr","attr","atten","atten","atten",]

for i in range(data[0].shape[1]):
    if plot_type[i] == 0:
        d = np.vstack([data[0][:,i,:], data[1][:,i,:], data[2][(data[2].shape[0] - 3):data[2].shape[0],i,:]]).flatten()
        q_h = np.quantile(d, 0.8)
        q_l = np.quantile(d, 0.2)
        d = np.clip(d, q_l, q_h)
        d_max = d.max()
        d_min = d.min()

        data[0][:,i,:] = NormalizeData(np.clip(data[0][:,i,:], q_l, q_h),d_min,d_max)
        data[1][:,i,:] = NormalizeData(np.clip(data[1][:,i,:], q_l, q_h),d_min,d_max)
        data[2][(data[2].shape[0] - 3):data[2].shape[0],i,:] = NormalizeData(np.clip(data[2][(data[2].shape[0] - 3):data[2].shape[0],i,:], q_l, q_h),d_min,d_max)

        for j in range(data[0].shape[0]):
            fig.add_trace(go.Box(y=np.concatenate((data[0][j,i,:], data[1][j,i,:])),name = methods[j], marker_color="#0059A0", showlegend=False, boxpoints=False), # model, explain, eval, n
                row=plot_row[i],
                col=plot_col[i],
            )        
        for j in range(data[2].shape[0] - 3,data[2].shape[0]):
            fig.add_trace(go.Box(y=data[2][j,i,:],name = methods[j], marker_color='#FF8777', showlegend=False, boxpoints=False), # model, explain, eval, n
                row=plot_row[i],
                col=plot_col[i],
            )

        fig.add_hline(y=np.median(np.concatenate((data[0][:,i,:], data[1][:,i,:]))),x0=0, x1=(1/17)*14, line_dash="dot", row=plot_row[i], col=plot_col[i], line_color="#000000", line_width=2)
        fig.add_hline(y=np.median(data[2][(data[2].shape[0] - 3) : data[2].shape[0],i,:]),x0=(1/17)*14, x1=1, line_dash="dot", row=plot_row[i], col=plot_col[i], line_color="#000000", line_width=2)

    elif plot_type[i] == 1:
        for j in range(data[0].shape[0]):
            fig.add_trace(go.Scatter(y=np.mean(np.concatenate((np.vstack(data[0][j,i,:]), np.vstack(data[1][j,i,:]))), axis = 0),name = methods[j],marker_color="#0059A0", showlegend=False),
                row=plot_row[i],
                col=plot_col[i],
            )

        for j in range(data[2].shape[0] - 3,data[2].shape[0]):
            fig.add_trace(go.Scatter(y =np.mean(np.vstack(data[2][j,i,:]), axis = 0),name = methods[j], marker_color='#FF8777', showlegend=False),
                row=plot_row[i],
                col=plot_col[i],
            )
    else:
        a = []
        b = []
        c = []
        for j in range(data[0].shape[0]):
            a.append([np.mean([
                    np.nan_to_num(pearsonr(
                        data[0][j,i,:][sample][len(data[0][j,i,:][0])-1],
                        data[0][j,i,:][sample][ix_patch],
                    )[0])
                    for ix_patch in range(len(data[0][j,i,:][0]))])
                    for sample in range(len(data[0][j,i,:]))
                ]
            )

            b.append([np.mean([
                    np.nan_to_num(pearsonr(
                        data[1][j,i,:][sample][len(data[1][j,i,:][0])-1],
                        data[1][j,i,:][sample][ix_patch],
                    )[0]) if np.sum(np.isnan(data[1][j,i,:][sample][ix_patch])) == 0 else 0
                    for ix_patch in range(len(data[1][j,i,:][0]))])
                    for sample in range(len(data[1][j,i,:]))
                ]
            )
        
            fig.add_trace(go.Box(y=np.concatenate((np.array(a[-1]), np.array(b[-1]))),name = methods[j], marker_color="#0059A0", showlegend=False, boxpoints=False), # model, explain, eval, n
                row=plot_row[i],
                col=plot_col[i],
            )        
            
        for j in range(data[2].shape[0] - 3,data[2].shape[0]):
            c.append([np.mean([
                np.nan_to_num(pearsonr(
                    data[2][j,i,:][sample][len(data[2][j,i,:][0])-1],
                    data[2][j,i,:][sample][ix_patch],
                )[0])
                for ix_patch in range(len(data[2][j,i,:][0]))])
                for sample in range(len(data[2][j,i,:]))
            ]
            )

            fig.add_trace(go.Box(y=np.array(c[-1]),name = methods[j], marker_color='#FF8777', showlegend=False, boxpoints=False), # model, explain, eval, n
                row=plot_row[i],
                col=plot_col[i],
            )

        fig.add_hline(y=np.median(np.concatenate((np.array(a), np.array(b)))),x0=0, x1=(1/17)*14, line_dash="dot", row=plot_row[i], col=plot_col[i], line_color="#000000", line_width=2)
        fig.add_hline(y=np.median(np.array(c)),x0=(1/17)*14, x1=1, line_dash="dot", row=plot_row[i], col=plot_col[i], line_color="#000000", line_width=2)

fig.update_yaxes(range = [0,1],row=3 if data[0].shape[1] == 20 else 2,col = 1 if data[0].shape[1] == 20 else 4)
fig.update_xaxes(range = [0,50],tickvals = [0,10,20,30,40,50], ticktext = ["0%","20%","40%","60%","80%","100%"],row=3 if data[0].shape[1] == 20 else 2,col = 1 if data[0].shape[1] == 20 else 4)

fig.update_layout(
    height=1000,
    width=2000,
    margin=dict(t=60, b=10, r=10, l=10),
    font=dict(
        family="Helvetica",
        color="#000000",
    ),
    title_font=dict(
        family="Helvetica",
        color="#000000",
    ),
    title={
        'text': "Evaluation Score Distributions for ImageNet Dataset per XAI Method and grouped into Attribution and Attention",
        # 'y':0.9,
        'x':0.0181,
        }

)

#fig = left_align_facet_plot_titles(fig)
fig.update_annotations(font_size=12)
#fig.write_image(os.getcwd() + "/data/figures/eval_distr/shapenet.png", scale=2)
fig.show()


/home/l727n/anaconda3/envs/xai-eval/lib/python3.9/site-packages/numpy/core/_methods.py:114: RuntimeWarning:

invalid value encountered in clip

/home/l727n/anaconda3/envs/xai-eval/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



### Ranking Computation

### Ranking Table

### Ranking Visualization